In [1]:
pip install requests beautifulsoup4 pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

In [5]:
base_url = "https://www.amazon.in/s?k=women+clothing&crid=UX6IQ6SOP3IE&sprefix=women+clo%2Caps%2C198&ref=nb_sb_ss_mvt-t11-ranker_4_9"


In [6]:
all_products = []


In [8]:
for page in range(1, 6):
    print(f"Scraping page {page}...")
    url = base_url.format(page)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    products = soup.find_all("div", {"data-component-type": "s-search-result"})

    for product in products:
        # Title
        title = product.h2.text.strip() if product.h2 else None

        # Product URL
    if product.h2 and product.h2.a and "href" in product.h2.a.attrs:
        link = "https://www.amazon.in" + product.h2.a["href"]
    else:
        link = None
        # Price
        price_tag = product.find("span", class_="a-price-whole")
        price = price_tag.text.strip().replace(",", "") if price_tag else None

        # Rating
        rating_tag = product.find("span", class_="a-icon-alt")
        rating = rating_tag.text.strip().split()[0] if rating_tag else None

        # Reviews
        review_tag = product.find("span", class_="a-size-base")
        reviews = review_tag.text.strip().replace(",", "") if review_tag else None

        all_products.append({
            "Title": title,
            "Price": price,
            "Rating": rating,
            "Reviews": reviews,
            "Product URL": link
        })

    time.sleep(2)  # Be respectful to Amazon's server

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...


In [9]:
df = pd.DataFrame(all_products)

In [10]:
# Convert columns to proper datatypes
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")

In [11]:
df = df.dropna(subset=["Title"]).reset_index(drop=True)

In [12]:
df.to_excel("amazon_women_clothing.xlsx", index=False)
print("Data saved to amazon_women_clothing.xlsx")

✅ Data saved to amazon_women_clothing.xlsx
